#Prioritized Experience Replay
---
In this notebook I am using prioritized experience replay. The general idea is to learn mainly from experiences that we can learn a lot from. Assumption: If there is a large $TD_{error}$ (defined as $\delta_t = R + \gamma \max\limits_{a'} Q(S', \argmax\limits_{a'}Q(S', a', w^-), w) - q(S, a, w)$) it means, that we can learn a lot from this experience, so we sample it more often. The sample chance is proportional to the latest $TD_{error}$. The probability of selecting each sample follows the formula: $P(i) = \frac{p_i}{\sum_k{p_k}}$, where $p_t=|\delta_t|$

As the sampling is not uniform any more, we have to modify the update rule as well. The way we sample the experiences should match the underlying distribution they came from. In uniform sampling it is preserved by default, but with non-uniform sampling we need to introduce the sampling weights $(\frac{1}{N}*\frac{1}{P(i)})$ where $N$ is the size of the replay buffer. As a result, our update rule looks like this: $\Delta w = \alpha (\frac{1}{N}*\frac{1}{P(i)}) \delta_t \nabla_w Q(S, a, w)$

There are couple of modifications that make this idea even more useful:
1) Experiences with $TD_{error}=0$ will never be sampled, and want every experience to have a chance of being used. To ensure it we add a small positive value $\epsilon$ to each probability ($p_t=|\delta_t| + \epsilon$). In the code, this value is denoted as `per_prior_eps`.
2) We do want to control how often the experiences with high $TD_{error}$ are being used. We are doing it, but introducing additional parameter $\alpha$ (`per_alpha` in code) and then sampling with a probability $P(i) = \frac{p_i^\alpha}{\sum_k{p_k^\alpha}}$. If $\alpha$ is equal to 0, all experiences are sampled uniformly. If $\alpha$ is eual to 1, we are fully leveraging the prioritized experience. 
3) We introduce additional parameter $\beta$ to control how much we want to use weight sampling in the update rule. The final update rule looks like: $\Delta w = \alpha (\frac{1}{N}*\frac{1}{P(i)})^\beta \delta_t \nabla_w Q(S, a, w)$. Usually we set initial $\beta$ to a low value, and increase it in time up to 1. In code, it is implemented by using `per_beta_start` (initial $\beta$) and `per_beta_frames` (number of steps necessary to push $\beta$ to 1). $\beta$ values are being increased between `per_beta_start` and 1 linearly.


In [ ]:
# Import packages
import gym
import torch
import numpy as np
from collections import deque
import matplotlib.pyplot as plt
%matplotlib inline

import config

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print ('Device:', device)

In [ ]:
env = gym.make(config.ENVIRONMENT)

In [ ]:
from rainbow.per_agent import Agent

agent = Agent(
    state_size=env.observation_space.shape[0], 
    action_size=env.action_space.n,
    buffer_size = int(1e5),
    batch_size = 64,
    gamma = 0.99,
    lr = 5e-4,
    update_every = 4, # How often to update the network
    device=device,
    # PER parameters
    per_alpha = 0.2,
    per_beta_start = 0.4,
    per_beta_frames = 1e5,
    per_prior_eps = 1e-6,    
    )



In [ ]:
def train_agent(n_episodes=config.MAX_EPISODES, 
        max_t=config.MAX_TIMESTEPS, 
        eps_start=config.EPSILON_START, 
        eps_end=config.EPSILON_END, 
        eps_decay=config.EPSILON_DECAY,
        expected_reward = config.EXPECTED_REWARD,
        update_target_every = 4
):
    """Deep Q-Learning.
    Args:
        n_episodes (int): maximum number of training episodes
        max_t (int): maximum number of timesteps per episode
        eps_start (float): starting value of epsilon, for 
            epsilon-greedy action selection
        eps_end (float): minimum value of epsilon
        eps_decay (float): decay factor (per episode) 
            for decreasing epsilon
        expected_reward (float): finish when the average score
            is greater than this value
        upate_target_every (int): how often should the target 
            network be updated. Default: 1 (per every episode) 
    Returns:
        scores (list): list of scores from each episode
    """
    scores = []                        
    scores_window = deque(maxlen=100)  
    eps = eps_start                    
    for episode in range(1, n_episodes+1):
        state, info = env.reset()
        score = 0
        for t in range(max_t):
            
            action = agent.act(state, eps)
            next_state, reward, done, truncated, info = env.step(action)
            
            agent.step(state, action, reward, next_state, done)
            
            state = next_state
            score += reward
            if done:
                break 
        scores_window.append(score)       
        scores.append(score)
                
        eps = max(eps_end, eps_decay*eps) 
        
        if episode % update_target_every == 0:
            agent.target_hard_update()
        
        mean_score = np.mean(scores_window)
        print(f'\rEpisode {episode}\tAverage Score: {mean_score:.2f}', end="")
        if episode % 100 == 0:
            print(f'\rEpisode {episode}\tAverage Score: {mean_score:.2f}')
            agent.save('checkpoint.pth')
        if mean_score >= expected_reward:
            print(f'\nDone in {episode:d} episodes!\tAverage Score: {mean_score:.2f}')
            agent.save('checkpoint.pth')
            break
    return scores

### Train the agent


In [ ]:
scores = train_agent()

# plot the scores
fig = plt.figure()
ax = fig.add_subplot(111)
plt.plot(np.arange(len(scores)), scores)
plt.ylabel('Score')
plt.xlabel('Episode #')
plt.show()